In [1]:
import pandas as pd
import json

In [2]:
train_img_path = '../../dataset/train/*'
train_json_path = '../../dataset/train.json'

test_img_path = '../../dataset/test/*'
test_json_path = '../../dataset/test.json'

model_labeled_csv_path = '../mmdetection/work_dirs/faster_rcnn_res50/submission_latest.csv'

In [ ]:
# --train json 파일 확인

# with open(train_json_path, 'r') as f:
#     json_data = json.load(f)
# print(json.dumps(json_data, indent='\t'))

In [13]:
# --model이 예측한 csv 파일 확인 및 리스트에 정보 담기

df = pd.read_csv(model_labeled_csv_path)
df = df.dropna()
print(df.isnull().sum())

anns_list = list()
id = 0
for x in df.index:
    image_name = df.loc[x][1].split('/')[1]
    image_id = int(image_name.split('.')[0])
    information = list(df.loc[x][0].split())
    for i in range(0, len(information), 6):
        label = int(information[i])
        # score = float(information[i+1])
        xmin = round(float(information[i+2]), 1)
        ymin = round(float(information[i+3]), 1)
        xmax = round(float(information[i+4]), 1)
        ymax = round(float(information[i+5]), 1)
        w = round(xmax - xmin, 2)
        h = round(ymax - ymin, 2)
        area = round(w*h, 2)

        ann = dict()
        ann['image_id'] = image_id
        ann['category_id'] = label
        ann['area'] = area
        ann['bbox'] = [xmin, ymin, w, h]
        ann['iscrowd'] = 0
        ann['id'] = id
        id += 1
        anns_list.append(ann)

PredictionString    0
image_id            0
dtype: int64


In [14]:
# --pseudo label json 파일 만들기

save_file_path = '../../dataset/pseudo.json'
with open(test_json_path, 'r') as f:
    json_data = json.load(f)

json_data['annotations'] = anns_list

with open(save_file_path, 'w', encoding='utf-8') as file:
    json.dump(json_data, file, indent='\t')